## Importa funções para o pré-processamento

In [1]:
from pre_processamento import *

from dlisio import dlis
import pandas as pd
import numpy as np

# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

## Carrega os dados

In [2]:
nomes_arquivos = []     # Armazena os nomes dos arquivos .dlis
leituras_dlis = []      # Armazena as leituras dos arquivos .dlis
nomes_anp = []          # Armazena os nomes obtidos das leituras

for file in glob.glob(r'**/Dados-dlis' + "/*.dlis", recursive=True):
    # Salva o nome do arquivo
    nomes_arquivos.append(file)

    # Salva os dados da leitura
    leitura, *tail = dlis.load(f'{file}')
    leituras_dlis.append(leitura)

    # Salva o nome do poço
    nome = leitura.origins[0].well_name
    nomes_anp.append(nome)

In [3]:
print(nomes_arquivos)
print(nomes_anp)

['Dados-dlis\\1-brsa-551-se_brsa_raw.dlis', 'Dados-dlis\\1-brsa-574-se_brsa_raw.dlis', 'Dados-dlis\\1-brsa-595-se_brsa_raw-1.dlis', 'Dados-dlis\\1-brsa-605-se_brsa_raw.dlis', 'Dados-dlis\\1-brsa-659-se_brsa_raw.dlis']
['1-FSG-1-SE', '1-FSJQ-1-SE', '1-BRSA-595-SE', '1-BRSA-605-SE', '1-BRSA-659-SE']


## Cria dicionário para armazenar os dados e respectivos nomes

In [3]:
# Casa itens da lista 'nome_anp_abreviados' com os itens da lista 'leituras_dlis'
pares = zip(nomes_anp, leituras_dlis)

# Cria dicionário 'dli_dict'
dli_dict = dict(pares)
dli_dict

{'1-FSG-1-SE': LogicalFile(AIT_SONIC_TLD_MCFL_018PUP),
 '1-FSJQ-1-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-595-SE': LogicalFile(GEOLOAD.1),
 '1-BRSA-605-SE': LogicalFile(AIT_SONIC_TLD_MCFL_048PUC),
 '1-BRSA-659-SE': LogicalFile(GEOLOAD.1)}

## Separa TODAS as curvas presentes nos .dlis de cada poço

In [5]:
channels_dict = {}

for key, poco in dli_dict.items():
    channels_list = []
    for frame in poco.frames:
        channels = frame.channels
        channels_list.append([channel.name for channel in channels])
    channels_dict[key] = sum(channels_list, [])

## Concatena os arquivos CSV das curvas em um único arquivo

In [6]:
import os

pasta = "Curvas_CSV"

# Lista para armazenar os dataframes repectivos aos .csv
df_list = []

# Itera sobre a pasta Curvas_CSV
for arquivo in os.listdir(pasta):
    file_path = os.path.join(pasta, arquivo)

    # Lê o arquivo CSV como um DF
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatena os DFs ao longo do eixo das colunas
df_concat = pd.concat(df_list, axis=0, ignore_index=True)

# Salva o DF concatenado em um .csv
df_concat.to_csv("Curvas_CSV/curvas_pocos.csv", index=False)

print("Arquivo Curvas_CSV/curvas_pocos.csv criado com sucesso")

Arquivo Curvas_CSV/curvas_pocos.csv criado com sucesso


## Processa os 'frames' do poço 1-BRSA-551-SE

#### Quantidade de 'frames' do poço 1-BRSA-551-SE

In [4]:
dli_dict['1-FSG-1-SE'].frames

[Frame(60B),
 Frame(10B),
 Frame(5B),
 Frame(30B),
 Frame(120B),
 Frame(180B),
 Frame(20B),
 Frame(1B),
 Frame(2B)]

#### Cria um dicionário para armazenar os 'frames' 

In [5]:
frames_dict_551 = {}
poco = dli_dict['1-FSG-1-SE']

for frame in poco.frames:
    indice = poco.frames.index(frame)

    curves = frame.curves()

    frames_dict_551[indice] = curves

#### Cria um 'dataframe' para cada 'frame'

In [7]:
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'RHOZ', 'DRHO', 'BSZ', 'BS', 'HCAL', 'CAL', 'CALI', 'DCALI', 'DCAL', 'PE', 'DT', 'DTC', 'ILD', 'RILD', 'IEL', 'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90', 'AHF90', 'AFH90', 'LLD', 'RLLD', 'HDRS', 'HLLD', 'LL7', 'RLL7']

dataframes_551 = {}
indice_frame = 0

for frame in frames_dict_551.values():
    i = 0
    names = frame.dtype.names
    dict_frame = {}

    for name in names:
        if name in curvas_escolhidas:
            curvas = [tupla[i] for tupla in frame]

            dict_frame[name] = curvas
        i += 1

    dataframes_551[indice_frame] = pd.DataFrame(dict_frame)
    indice_frame += 1

In [18]:
dataframes_551

{0:           TDEP   BS         RT          GR      NPHI      DCAL          DT
 0     283800.0  8.5  16.035276  129.733124  0.691645 -5.699512   99.898071
 1     283740.0  8.5   1.603191  129.733124  0.691645 -5.699512   99.898071
 2     283680.0  8.5   1.601933  129.733124  0.691645 -5.699512   99.898071
 3     283620.0  8.5   1.599770  129.733124  0.691645 -5.699512   99.898071
 4     283560.0  8.5   1.594869  129.733124  0.691645 -5.699512  100.353745
 ...        ...  ...        ...         ...       ...       ...         ...
 4551   10740.0  8.5  15.466356   19.685118  0.684838 -5.745168   78.800926
 4552   10680.0  8.5  15.466356   20.519228  0.664183 -5.745168   78.800926
 4553   10620.0  8.5  15.466356   25.015450  0.690279 -5.745168   78.800926
 4554   10560.0  8.5  15.466356   24.739817  0.690279 -5.745168   78.800926
 4555   10500.0  8.5  15.466356   24.739817  0.690279 -5.745168   78.800926
 
 [4556 rows x 7 columns],
 1:            TDEP      HCAL
 0      283800.0  2.800488


In [36]:
frame = frames_dict_551[0]

primeira_coluna = [tupla[1] for tupla in frame]

primeira_coluna

[283800.0,
 283740.0,
 283680.0,
 283620.0,
 283560.0,
 283500.0,
 283440.0,
 283380.0,
 283320.0,
 283260.0,
 283200.0,
 283140.0,
 283080.0,
 283020.0,
 282960.0,
 282900.0,
 282840.0,
 282780.0,
 282720.0,
 282660.0,
 282600.0,
 282540.0,
 282480.0,
 282420.0,
 282360.0,
 282300.0,
 282240.0,
 282180.0,
 282120.0,
 282060.0,
 282000.0,
 281940.0,
 281880.0,
 281820.0,
 281760.0,
 281700.0,
 281640.0,
 281580.0,
 281520.0,
 281460.0,
 281400.0,
 281340.0,
 281280.0,
 281220.0,
 281160.0,
 281100.0,
 281040.0,
 280980.0,
 280920.0,
 280860.0,
 280800.0,
 280740.0,
 280680.0,
 280620.0,
 280560.0,
 280500.0,
 280440.0,
 280380.0,
 280320.0,
 280260.0,
 280200.0,
 280140.0,
 280080.0,
 280020.0,
 279960.0,
 279900.0,
 279840.0,
 279780.0,
 279720.0,
 279660.0,
 279600.0,
 279540.0,
 279480.0,
 279420.0,
 279360.0,
 279300.0,
 279240.0,
 279180.0,
 279120.0,
 279060.0,
 279000.0,
 278940.0,
 278880.0,
 278820.0,
 278760.0,
 278700.0,
 278640.0,
 278580.0,
 278520.0,
 278460.0,
 278400.0,

#### Quantidade de 'frames' no poço 1-BRSA-605-SE

In [10]:
dli_dict['1-BRSA-605-SE'].frames

[Frame(60B), Frame(10B), Frame(30B), Frame(20B)]

#### Cria um 'dataframe' para cada 'frame'

In [11]:
dataframes_605 = {}
poco = dli_dict['1-BRSA-605-SE']

for frame in poco.frames:
    indice = poco.frames.index(frame)

    curves = pd.DataFrame(frame.curves())

    dataframes_605[indice] = curves

In [8]:
for value in dataframes_605.values():
    value["TDEP"] = value["TDEP"] / 10

#### Converte de polegada para metro

In [9]:
for value in dataframes_605.values():
    value["TDEP"] = value["TDEP"] * 0.0254

#### Salva os dataframes como arquivos CSV

In [8]:
for key, value in dataframes_605.items():
    file_name = f"Frames_605/frame_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Frames_605/frame_0.csv criado com sucesso.
Arquivo Frames_605/frame_1.csv criado com sucesso.
Arquivo Frames_605/frame_2.csv criado com sucesso.
Arquivo Frames_605/frame_3.csv criado com sucesso.


#### Remove valores de profundidade que não estão presentes no primeiro frame

In [10]:
for key, value in dataframes_605.items():
    dataframes_605[key] = value[value["TDEP"].isin(dataframes_605[0]["TDEP"])]

#### Junta os 4 dataframes 

In [11]:
dataframes_605_merged = dataframes_605[0]

for i in range(1, len(dataframes_605)):
    dataframes_605_merged = pd.merge(dataframes_605_merged, dataframes_605[i], on='TDEP', how='outer', suffixes=(None, "_new"))

#### Mantém apenas as curvas escolhidas

In [12]:
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'RHOZ', 'DRHO', 'BSZ', 'BS', 'HCAL', 'CAL', 'CALI', 'DCALI', 'DCAL', 'PE', 'DT', 'DTC', 'ILD', 'RILD', 'IEL', 'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90', 'AHF90', 'AFH90', 'LLD', 'RLLD', 'HDRS', 'HLLD', 'LL7', 'RLL7']

# Remove colunas duplicadas 
dataframes_605_merged = dataframes_605_merged.loc[:, ~dataframes_605_merged.columns.duplicated()]

# Filtra de modo a manter apenas as curvas escolhidas
dataframes_605_merged_filtered = dataframes_605_merged.filter(items=curvas_escolhidas)

In [14]:
dlis_df_dict = {}   # Conterá os dataframes respectivos aos poços

# Curvas de perfis escolhidas
curvas_escolhidas = ['TDEP', 'GR', 'NPHI', 'RHOB', 'RHOZ', 'DRHO', 'BSZ', 'BS', 'HCAL', 'CAL', 'CALI', 'DCALI', 'DCAL', 'PE', 'DT', 'DTC', 'ILD', 'RILD', 'IEL', 'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90', 'AHF90', 'AFH90', 'LLD', 'RLLD', 'HDRS', 'HLLD', 'LL7', 'RLL7']


# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for chave, poco in dli_dict.items():

    # Armazenando as curvas que serão utilizadas em uma lista
    curvas_utilizadas = [
        channel.name                                    # Os elementos da lista serão os nomes das curvas
        for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
        if channel.name in curvas_escolhidas            # As curvas que não utilizaremos não serão armazenadas na lista
    ]
    conjunto_aux = set(curvas_utilizadas)
    curvas_utilizadas_sem_duplicados = list(conjunto_aux)
    
    #frames = [np.atleast_1d(frame) for frame in poco.frames]
    #curvas = np.concatenate(frames, axis=0)
    try:
        dataframe = pd.DataFrame()
        
        for frame in poco.frames:

            curvas = frame.curves()

            dataframe = pd.concat([dataframe, pd.DataFrame(curvas[curvas_utilizadas_sem_duplicados])])

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df_dict[chave] = dataframe
    except:
        pass


In [15]:
dlis_df_dict['1-BRSA-605-SE'] = dataframes_605_merged_filtered
dlis_df_dict.keys()

dict_keys(['1-FSJQ-1-SE', '1-BRSA-595-SE', '1-BRSA-659-SE', '1-BRSA-605-SE'])

## Transforma os valores -999.25 em nulos

In [16]:
for poco in dlis_df_dict.values():
    poco.replace([-999.25], [None], inplace = True)

## Aplicando os mnemônicos

In [17]:
aplica_mnemonico(dlis_df_dict, ['BS', 'BSZ'], 'BS')
aplica_mnemonico(dlis_df_dict, ['LLD',	'LL7',	'RLLD',	 'RLL7', 'HDRS', 'HLLD', 'ILD',	'RILD',	'IEL',	'AIT90', 'AHT90', 'RT90', 'AT90', 'AO90', 'RT', 'AF90',	'AHF90', 'AFH90'], 'RESD')
aplica_mnemonico(dlis_df_dict, ['RHOB', 'RHOZ'], 'RHOB')
aplica_mnemonico(dlis_df_dict, ['DTC', 'DT'], 'DT')
aplica_mnemonico(dlis_df_dict, ['HCAL', 'CAL', 'CALI'], 'CAL')
aplica_mnemonico(dlis_df_dict, ['DCAL', 'DCALI'], 'DCAL')
aplica_mnemonico(dlis_df_dict, ['DRHO', 'HDRA'], 'DRHO')

## Adiciona coluna DCAL

In [18]:
add_DCAL(dlis_df_dict)

## Preenche os poços com curvas faltando

In [19]:
# Se um dos poços não tiver uma dessas curvas, adicionamos a coluna da curva e mantemos os valores como None
curvas_obrigatorias = ['TDEP', 'BS', 'CAL', 'DCAL', 'GR', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

# Percorre todos os poços
for poco in dlis_df_dict.values():
    # Percorre todas as curvas obrigatórias
    for curva in curvas_obrigatorias:
        # Se o poço não tiver a curva
        if curva not in poco.columns:
            # Adiciona a coluna e os valores dela = None
            poco[curva] = None

## Temos as seguintes curvas 

In [20]:
for key, poco in dlis_df_dict.items():
    curvas = sorted(poco.keys())
    print(f"{key}: {curvas}")

1-FSJQ-1-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-595-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-659-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']
1-BRSA-605-SE: ['BS', 'CAL', 'DCAL', 'DRHO', 'DT', 'GR', 'NPHI', 'PE', 'RESD', 'RHOB', 'TDEP']


## Remove valores DRHO e DCAL indesejados (Só depois)

In [ ]:
#limita_curva(dlis_df_dict, "DRHO", -0.15, 0.15)
#limita_curva(dlis_df_dict, "DCAL", -2, 2)

## Inverte a ordem das linhas dos dataframes

In [21]:
for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].iloc[::-1]

## Ordena as colunas dos dataframes

In [22]:
# Simon e Vandelli vao definir a ordem de preferencia
ordem_desejada = ['TDEP', 'BS', 'CAL', 'DCAL', 'GR', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

for key in dlis_df_dict.keys():
    dlis_df_dict[key] = dlis_df_dict[key].reindex(columns=ordem_desejada)

## Salva os dados dos dataframes em arquivos CSV

In [23]:
for key, value in dlis_df_dict.items():
    file_name = f"Pocos-pre-processados/poco_{key}.csv"
    value.to_csv(file_name, index=False)
    print(f"Arquivo {file_name} criado com sucesso.")

Arquivo Pocos-pre-processados/poco_1-FSJQ-1-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-595-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-659-SE.csv criado com sucesso.
Arquivo Pocos-pre-processados/poco_1-BRSA-605-SE.csv criado com sucesso.
